In [1]:
# นำเข้า pandas สำหรับการจัดการข้อมูลแบบ DataFrame

import pandas as pd

# ตั้งค่าการแสดงผลของ pandas ให้แสดงผลครบทุก row และ column ไม่มีการย่อหน้า

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# นำเข้า SparkSession สำหรับการใช้งาน Spark

from pyspark.sql import SparkSession

In [3]:
# ตั้งค่าการทำงานของ Spark และสร้าง SparkSession

spark = SparkSession.\
        builder.\
        appName("Static Dataframe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        getOrCreate()

23/06/21 11:55:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# อ่านข้อมูลจากไฟล์ CSV และสร้าง DataFrame ด้วย Spark
# โปรดระวัง ท่านจะต้อง manual download ไฟล์ 2023-01-01-2023-04-30.csv มา manual upload เข้า jupyter
# link สำหรับไฟล์ฯ คือ 
# https://github.com/aekanun2020/2023-IoTAnalytics-and-PowerBI/blob/main/OpenAQ_Data/2023-01-01-2023-04-30.csv

raw_df = spark.read.option('header','true').csv('./2023-01-01-2023-04-30.csv')

In [5]:
# นับจำนวนแถวของ DataFrame

raw_df.count()

930

In [6]:
# ใช้ command line สำหรับนับจำนวนบรรทัดของไฟล์ CSV

! wc -l 2023-01-01-2023-04-30.csv

931 2023-01-01-2023-04-30.csv


In [7]:
# แสดง schema ของ DataFrame
# ซึ่งเราจะเห็นว่า มี col. ที่ data type ไม่สอดคล้องกับความหมายทางธุรกิจ (Type Mismatch)

raw_df.printSchema()

root
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [8]:
# ดูแถวแรกของ DataFrame
# ซึ่งเราจะเห็นว่า ข้อมูลใน col. date มีโครงสร้างแบบ JSON


raw_df.take(1)

[Row(locationId='225577', location='Bank of Ayuthaya Head Office Yan Nawa, Bangkok', parameter='pm25', value='36.0', date="{'utc': '2022-12-31T18:00:00+00:00', 'local': '2023-01-01T01:00:00+07:00'}", unit='µg/m³', coordinates="{'latitude': 13.679226, 'longitude': 100.54687}", country='NA', city=None, isMobile='False', isAnalysis=None, entity='Governmental Organization', sensorType='reference grade')]

In [9]:
# นำเข้าฟังก์ชันที่จำเป็นสำหรับการแปลงข้อมูล

from pyspark.sql.functions import get_json_object
from pyspark.sql.types import TimestampType, FloatType
from pyspark.sql import functions as sparkf

In [10]:
# แยกสตริงออกมาเป็นคีย์และค่าของ dictionary และแปลงค่าที่ได้ให้เป็น timestamp รวมถึง แปลงค่าฝุ่น (value) เป็น float

final_df = raw_df.withColumn("date_utc", get_json_object(sparkf.col('date'), '$.utc'))\
.withColumn("date_local", get_json_object(sparkf.col('date'), '$.local'))\
.withColumn("date_utc", sparkf.col('date_utc').cast(TimestampType()))\
.withColumn("date_local", sparkf.col('date_local').cast(TimestampType()))\
.withColumn("value", sparkf.col('value').cast(FloatType()))

In [11]:
# แสดงข้อมูล column date_utc และ date_local
# ซึ่งทั้ง 2 col. นี้ควรจะตรงกัน เนื่องจากทั้ง 2 col. มี type เป็น timestamp

final_df.select("date_utc", "date_local").show()

+-------------------+-------------------+
|           date_utc|         date_local|
+-------------------+-------------------+
|2022-12-31 18:00:00|2022-12-31 18:00:00|
|2022-12-31 19:00:00|2022-12-31 19:00:00|
|2022-12-31 20:00:00|2022-12-31 20:00:00|
|2022-12-31 21:00:00|2022-12-31 21:00:00|
|2022-12-31 22:00:00|2022-12-31 22:00:00|
|2022-12-31 23:00:00|2022-12-31 23:00:00|
|2023-01-01 00:00:00|2023-01-01 00:00:00|
|2023-01-01 01:00:00|2023-01-01 01:00:00|
|2023-01-01 02:00:00|2023-01-01 02:00:00|
|2023-01-01 03:00:00|2023-01-01 03:00:00|
|2023-01-01 04:00:00|2023-01-01 04:00:00|
|2023-01-01 05:00:00|2023-01-01 05:00:00|
|2023-01-01 06:00:00|2023-01-01 06:00:00|
|2023-01-01 07:00:00|2023-01-01 07:00:00|
|2023-01-01 08:00:00|2023-01-01 08:00:00|
|2023-01-01 09:00:00|2023-01-01 09:00:00|
|2023-01-01 10:00:00|2023-01-01 10:00:00|
|2023-01-01 11:00:00|2023-01-01 11:00:00|
|2023-01-01 12:00:00|2023-01-01 12:00:00|
|2023-01-01 13:00:00|2023-01-01 13:00:00|
+-------------------+-------------

In [12]:
# แสดง schema ของ DataFrame ที่ถูกแปลงค่าแล้ว

final_df.printSchema()

root
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: float (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- date_utc: timestamp (nullable = true)
 |-- date_local: timestamp (nullable = true)



In [13]:
# ลงทะเบียน DataFrame เป็น TempTable เพื่อให้สามารถทำความสะอาดข้อมูลได้

final_df.registerTempTable('pm2022')

In [14]:
# สร้างสถิติเบื้องต้นของ DataFrame และแปลงเป็น pandas DataFrame ก่อนทำ transpose สำหรับแสดงผลที่ดีขึ้น

final_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
locationId,930,225577.0,0.0,225577,225577
location,930,None,None,"Bank of Ayuthaya Head Office Yan Nawa, Bangkok","Bank of Ayuthaya Head Office Yan Nawa, Bangkok"
parameter,930,None,None,pm25,pm25
value,930,43.94516129032258,15.125602701326343,14.0,99.0
date,930,None,None,"{'utc': '2022-12-31T18:00:00+00:00', 'local': '2023-01-01T01:00:00+07:00'}","{'utc': '2023-04-30T16:00:00+00:00', 'local': '2023-04-30T23:00:00+07:00'}"
unit,930,None,None,µg/m³,µg/m³
coordinates,930,None,None,"{'latitude': 13.679226, 'longitude': 100.54687}","{'latitude': 13.679226, 'longitude': 100.54687}"
country,930,None,None,NA,NA
city,0,None,None,None,None


In [15]:
# ใช้ Spark SQL เพื่อสร้าง DataFrame ใหม่ที่ group by ตาม value และ window ของ date_utc ที่จัดเป็นช่วง 360 นาที และเรียงลำดับตาม window และ value

result_df = spark.sql('SELECT value, WINDOW(date_utc, "360 minutes"), COUNT(*) \
FROM pm2022 GROUP BY value, WINDOW(date_utc, "360 minutes") ORDER BY window, value')


In [16]:
# แสดง schema ของ DataFrame ที่สร้างใหม่

result_df.printSchema()

root
 |-- value: float (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- count(1): long (nullable = false)



In [17]:
# แปลง DataFrame เป็น pandas DataFrame และแสดงผล

result_df.toPandas()

,value,window,count(1)
0,36.0,"(2022-12-31 18:00:00, 2023-01-01 00:00:00)",6
1,30.0,"(2023-01-01 00:00:00, 2023-01-01 06:00:00)",2
2,31.0,"(2023-01-01 00:00:00, 2023-01-01 06:00:00)",2
3,33.0,"(2023-01-01 00:00:00, 2023-01-01 06:00:00)",1
4,34.0,"(2023-01-01 00:00:00, 2023-01-01 06:00:00)",1
5,29.0,"(2023-01-01 06:00:00, 2023-01-01 12:00:00)",6
6,29.0,"(2023-01-01 12:00:00, 2023-01-01 18:00:00)",2
7,30.0,"(2023-01-01 12:00:00, 2023-01-01 18:00:00)",4
8,31.0,"(2023-01-01 18:00:00, 2023-01-02 00:00:00)",4
9,32.0,"(2023-01-01 18:00:00, 2023-01-02 00:00:00)",2
